In [ ]:
import os

# Manual credentials entry
os.environ['KAGGLE_USERNAME'] = "srijaniss" # Replace with your username
# os.environ['KAGGLE_KEY'] = "KGAT_55f46f799d55bc98210eec17be0b7503"       # Replace with your API key
os.environ['KAGGLE_KEY'] = "KGAT_5aa6e2154b92ea00f930dfa0a51df616"
# Verify the connection by listing datasets
# !kaggle datasets list

In [ ]:
# Download the dataset
!kaggle datasets download -d ealaxi/paysim1

# Unzip it
!unzip paysim1.zip

In [ ]:
import pandas as pd

# Load the data
df = pd.read_csv('PS_20174392719_1491204439457_log.csv')

# Look at the first few rows
df.head()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset (assuming you named the file 'payments.csv')
df = pd.read_csv('PS_20174392719_1491204439457_log.csv')

# Look at the first 5 rows and column names
print(df.head())
print(df.info())

In [ ]:
fraud_counts = df['isFraud'].value_counts()
print(fraud_counts)

In [ ]:
type_counts = df['type'].value_counts()
print(type_counts)

In [ ]:
# Calculate the percentage
fraud_pct = (fraud_counts[1] / len(df)) * 100
print(f"Percentage of Fraud: {fraud_pct:.2f}%")

In [ ]:
# Visualize it
sns.countplot(x='isFraud', data=df)
plt.title('Distribution of Legit vs Fraud Transactions')
plt.show()

Finding the HOTSPOT


In [ ]:
fraud_by_type = df.groupby('type')['isFraud'].sum()
print(fraud_by_type)

In [ ]:
# Visualize it to see the 'Hotspots'
plt.figure(figsize=(6,2))
fraud_by_type.plot(kind='bar', color='salmon')
plt.title('Fraud Cases by Transaction Type')
plt.ylabel('Number of Fraudulent Hits')
plt.show()

In [ ]:
# Compare the 'amount' for Fraud vs Legit
plt.figure(figsize=(6,2))
sns.boxplot(x='isFraud', y='amount', data=df)
plt.title('Transaction Amount: Legit vs Fraud')
plt.ylim(0, 2000000) # Zooming in to see the boxes clearly
plt.show()

In [ ]:
# Filter data to only include TRANSFER and CASH_OUT
relevant_types = ['TRANSFER', 'CASH_OUT']
df_filtered = df[df['type'].isin(relevant_types)]

print(f"Original rows: {len(df)}")
print(f"Filtered rows: {len(df_filtered)}")

In [ ]:
# Dropping columns that aren't useful for numerical patterns
# 'isFlaggedFraud' is an old system we are trying to replace, so we drop it too.
cols_to_drop = ['nameOrig', 'nameDest', 'isFlaggedFraud']
df_cleaned = df_filtered.drop(columns=cols_to_drop)

print(df_cleaned.head())

In [ ]:
# Simple manual encoding: CASH_OUT = 0, TRANSFER = 1
df_cleaned['type'] = df_cleaned['type'].map({'CASH_OUT': 0, 'TRANSFER': 1})

# Check if 'type' is now numeric
print(df_cleaned['type'].value_counts())

In [ ]:
# X = everything except the 'isFraud' column
X = df_cleaned.drop('isFraud', axis=1)

# y = only the 'isFraud' column
y = df_cleaned['isFraud']

print("Features (X) shape:", X.shape)
print("Target (y) shape:", y.shape)

In [ ]:
# Step 4: Split the data
from sklearn.model_selection import train_test_split

# This line creates y_test!
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Predict on the test data
raw_predictions = model.predict(X_test)

# Convert predictions: -1 (anomaly) becomes 1 (fraud), 1 (normal) becomes 0 (legit)
y_pred = [1 if x == -1 else 0 for x in raw_predictions]

print("First 10 predictions:", y_pred[:10])

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# --- RE-DEFINING DATA (Ensures X and y exist) ---
X = df_cleaned.drop('isFraud', axis=1)
y = df_cleaned['isFraud']

# --- STEP 4: SPLITTING & TRAINING ---
# This creates X_train, X_test, y_train, and y_test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the model
model = IsolationForest(n_estimators=100, contamination=0.01, random_state=42)

# FIT THE MODEL (This fixes the NotFittedError)
model.fit(X_train)

# --- STEP 5: PREDICTING & EVALUATING ---
# Predict on the test data
raw_predictions = model.predict(X_test)

# Convert predictions: -1 to 1 (fraud), 1 to 0 (legit)
y_pred = [1 if x == -1 else 0 for x in raw_predictions]

# --- VISUALIZE RESULTS ---
print("Model Performance Report:")
print(classification_report(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Final Confusion Matrix')
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt

# Generate the confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Visualize it using Seaborn
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Reds')
plt.title('Confusion Matrix: Predicted vs Actual')
plt.ylabel('Actual Label (Fraud or Not)')
plt.xlabel('Predicted Label (Model\'s Guess)')
plt.show()